In [14]:
%load_ext autoreload
%autoreload 2

In [1]:
!pip install transformer_lens
!pip uninstall jax jaxlib -y
!pip install jax jaxlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.9/109.9 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 11.6 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 72.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 79.7 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 40.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.12.0
    Uninstalling huggingface-hub-0.12.0:
      Successfully uninstalled huggingface-hub-0.12.0
  Attempting uninstall: wandb
    Found existing installation: wandb 0.13.4
    Uninstalling wandb-0.13.4:
      Successfully uninstalled wandb-0.13.4
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.1

In [1]:
from transformer_lens import HookedTransformer
import torch

import json
import time

from src.neuron_texts import find_truncated_texts

In [2]:
# Load jsons from ../experiment_data/1_next_token_neurons
filename = "2023-10-14_17-23-35_gpt2-large_mid_test"
with open(f'./experiment_data/2_max_activating_texts/{filename}.json') as f:
    max_text_data = json.load(f)

In [3]:
!ls experiment_data/2_max_activating_texts/

2023-10-11_14-06-30_gpt2-small.json   2023-10-11_16-36-33_gpt2-large.json
2023-10-11_14-29-40_gpt2-medium.json  2023-10-13_03-52-29_gpt2-small_test.json
2023-10-11_14-30-39_gpt2-medium.json  2023-10-13_04-29-47_gpt2-medium_test.json


In [3]:
# Parameters
parameters = max_text_data['parameters']
model_name = parameters['model_name']

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [4]:
model = HookedTransformer.from_pretrained(
    model_name,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
    device=device,
)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-large into HookedTransformer


In [5]:
# from datasets import load_dataset
# dataset = load_dataset("NeelNanda/pile-10k", split="train")
# dataset_text_list = [x['text'] for x in dataset]

import pickle

with open('./text_list_dict_test.pkl', 'rb') as f:
    dataset_text_dict = pickle.load(f)

dataset_text_list = [x['text'] for x in dataset_text_dict]
dataset = dataset_text_dict

In [6]:
neuron_20_examples = find_truncated_texts(
    model=model,
    neuron_max_acts=max_text_data['neuron_max_acts'],
    dataset=dataset,
    device=device,
    num_samples=30,
    act_ratio=0.8,
)

30it [00:25,  1.15it/s][00:00<?, ?it/s]
30it [00:29,  1.03it/s][00:25<42:53, 25.99s/it]
30it [00:30,  1.00s/it][00:55<45:33, 27.89s/it]
30it [00:24,  1.20it/s][01:25<46:39, 28.86s/it]
30it [00:28,  1.05it/s][01:50<43:41, 27.31s/it]
30it [00:23,  1.25it/s][02:18<43:53, 27.72s/it]
30it [00:26,  1.13it/s][02:42<41:32, 26.52s/it]
30it [00:25,  1.16it/s][03:09<41:08, 26.54s/it]
30it [00:28,  1.06it/s][03:35<40:24, 26.35s/it]
30it [00:27,  1.07it/s][04:03<40:54, 26.97s/it]
30it [00:25,  1.18it/s] [04:31<40:56, 27.29s/it]
30it [00:26,  1.12it/s] [04:57<39:36, 26.71s/it]
30it [00:28,  1.03it/s] [05:24<39:22, 26.84s/it]
30it [00:24,  1.23it/s] [05:53<39:52, 27.50s/it]
30it [00:17,  1.69it/s] [06:17<38:06, 26.59s/it]
30it [00:17,  1.75it/s] [06:35<33:53, 23.93s/it]
30it [00:29,  1.03it/s] [06:52<30:38, 21.88s/it]
30it [00:24,  1.22it/s] [07:21<33:15, 24.04s/it]
30it [00:27,  1.10it/s] [07:46<33:05, 24.22s/it]
30it [00:26,  1.11it/s] [08:13<33:57, 25.16s/it]
30it [00:29,  1.02it/s] [08:40<34:16, 

In [ ]:
output = {
    'parameters': parameters,
    'neuron_to_trunc_data': neuron_20_examples,
    'prior_filename': filename,
}

# Save json to ../experiment_data/2_max_activating_texts
timestamp = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime(int(time.time())))
new_filename = f"{timestamp}_{model_name}_mid_test.json"

with open(f'./experiment_data/3_trimmed_texts/{new_filename}', 'w') as f:
    json.dump(output, f)